# 第5章: 回帰分析

この章では、simple-statisticsとmathjsを使って回帰分析を学びます。

## 学習目標
- 単回帰分析（最小二乗法）
- 回帰モデルの評価
- 重回帰分析
- 非線形回帰（多項式回帰、指数回帰）
- ロバスト回帰

In [1]:
// ライブラリの読み込み
const math = await import('https://esm.sh/mathjs@13.0.0');
const ss = await import('https://esm.sh/simple-statistics@7.8.8');
console.log('Libraries loaded!');

Libraries loaded!


## 5.1 単回帰分析

単回帰分析は、1つの説明変数（x）から目的変数（y）を予測するモデルです。

$$y = \beta_0 + \beta_1 x + \epsilon$$

- $\beta_0$: 切片（intercept）
- $\beta_1$: 傾き（slope）
- $\epsilon$: 誤差項

### 5.1.1 最小二乗法の原理

In [2]:
// サンプルデータ（勉強時間と試験の点数）
const studyHours = [1, 2, 3, 4, 5, 6, 7, 8];
const testScores = [52, 58, 65, 70, 75, 82, 88, 95];

console.log('=== 最小二乗法 ===');
console.log('勉強時間 (x):', studyHours);
console.log('試験の点数 (y):', testScores);
console.log('');

// simple-statisticsの線形回帰
const regression = ss.linearRegression(studyHours.map((x, i) => [x, testScores[i]]));
console.log('回帰係数:');
console.log('  傾き (β₁):', regression.m.toFixed(4));
console.log('  切片 (β₀):', regression.b.toFixed(4));
console.log('');
console.log('回帰式: y =', regression.b.toFixed(2), '+', regression.m.toFixed(2), '× x');

=== 最小二乗法 ===


勉強時間 (x): [
  1, 2, 3, 4,
  5, 6, 7, 8
]


試験の点数 (y): [
  52, 58, 65, 70,
  75, 82, 88, 95
]


回帰係数:


  傾き (β₁): 6.0357


  切片 (β₀): 45.9643


回帰式: y = 45.96 + 6.04 × x


### 5.1.2 最小二乗法の計算過程

In [3]:
// 最小二乗法を手動で計算
const x = studyHours;
const y = testScores;
const n = x.length;

const xMean = ss.mean(x);
const yMean = ss.mean(y);

console.log('=== 計算過程 ===');
console.log('n =', n);
console.log('x̄ =', xMean);
console.log('ȳ =', yMean);
console.log('');

// β₁ = Σ(xᵢ - x̄)(yᵢ - ȳ) / Σ(xᵢ - x̄)²
let numerator = 0;
let denominator = 0;

console.log('偏差の計算:');
x.forEach((xi, i) => {
    const yi = y[i];
    const devX = xi - xMean;
    const devY = yi - yMean;
    const product = devX * devY;
    const devXSquared = devX * devX;
    
    numerator += product;
    denominator += devXSquared;
    
    console.log(`  i=${i+1}: (${xi}-${xMean})(${yi}-${yMean}) = ${product.toFixed(2)}, (${xi}-${xMean})² = ${devXSquared.toFixed(2)}`);
});

const beta1 = numerator / denominator;
const beta0 = yMean - beta1 * xMean;

console.log('');
console.log('β₁ =', numerator.toFixed(2), '/', denominator.toFixed(2), '=', beta1.toFixed(4));
console.log('β₀ =', yMean, '-', beta1.toFixed(4), '×', xMean, '=', beta0.toFixed(4));

=== 計算過程 ===


n = 8


x̄ = 4.5


ȳ = 73.125


偏差の計算:


...

  i=6: (6-4.5)(82-73.125) = 13.31, (6-4.5)² = 2.25


  i=7: (7-4.5)(88-73.125) = 37.19, (7-4.5)² = 6.25


  i=8: (8-4.5)(95-73.125) = 76.56, (8-4.5)² = 12.25


β₁ = 253.50 / 42.00 = 6.0357


β₀ = 73.125 - 6.0357 × 4.5 = 45.9643


### 5.1.3 予測値の計算

In [4]:
// 予測関数の作成
const predict = ss.linearRegressionLine(regression);

console.log('=== 予測値と残差 ===');
console.log('');

let residualSum = 0;
let residualSquaredSum = 0;

x.forEach((xi, i) => {
    const yi = y[i];
    const predicted = predict(xi);
    const residual = yi - predicted;
    residualSum += residual;
    residualSquaredSum += residual * residual;
    
    console.log(`x=${xi}: 実測値=${yi}, 予測値=${predicted.toFixed(2)}, 残差=${residual.toFixed(2)}`);
});

console.log('');
console.log('残差の合計:', residualSum.toFixed(4), '(≈ 0になる)');
console.log('残差平方和 (RSS):', residualSquaredSum.toFixed(4));

// 新しい値での予測
console.log('');
console.log('予測例:');
console.log('  10時間勉強したら:', predict(10).toFixed(2), '点');
console.log('  5.5時間勉強したら:', predict(5.5).toFixed(2), '点');

=== 予測値と残差 ===


x=1: 実測値=52, 予測値=52.00, 残差=0.00


x=2: 実測値=58, 予測値=58.04, 残差=-0.04


x=3: 実測値=65, 予測値=64.07, 残差=0.93


x=4: 実測値=70, 予測値=70.11, 残差=-0.11


...

残差の合計: -0.0000 (≈ 0になる)


残差平方和 (RSS): 2.8214


予測例:


  10時間勉強したら: 106.32 点


  5.5時間勉強したら: 79.16 点


## 5.2 回帰モデルの評価

### 5.2.1 決定係数（R²）

In [5]:
// R²の計算
console.log('=== 決定係数 R² ===');
console.log('');

// 全変動（TSS: Total Sum of Squares）
let tss = 0;
y.forEach(yi => {
    tss += Math.pow(yi - yMean, 2);
});

// 残差変動（RSS: Residual Sum of Squares）
let rss = 0;
x.forEach((xi, i) => {
    const yi = y[i];
    const predicted = predict(xi);
    rss += Math.pow(yi - predicted, 2);
});

// 回帰変動（ESS: Explained Sum of Squares）
let ess = 0;
x.forEach((xi) => {
    const predicted = predict(xi);
    ess += Math.pow(predicted - yMean, 2);
});

console.log('全変動 (TSS):', tss.toFixed(4));
console.log('回帰変動 (ESS):', ess.toFixed(4));
console.log('残差変動 (RSS):', rss.toFixed(4));
console.log('');
console.log('TSS = ESS + RSS の確認:', (ess + rss).toFixed(4));
console.log('');

// R² = ESS / TSS = 1 - RSS / TSS
const rSquared = 1 - rss / tss;
console.log('R² = 1 - RSS/TSS =', rSquared.toFixed(4));
console.log('');
console.log('解釈: 勉強時間で点数の変動の', (rSquared * 100).toFixed(1) + '%を説明できる');

// simple-statisticsでの計算
const rSquaredSS = ss.rSquared(x.map((xi, i) => [xi, y[i]]), predict);
console.log('\nsimple-statisticsでの計算: R² =', rSquaredSS.toFixed(4));

=== 決定係数 R² ===


全変動 (TSS): 1532.8750


回帰変動 (ESS): 1530.0536


残差変動 (RSS): 2.8214


TSS = ESS + RSS の確認: 1532.8750


R² = 1 - RSS/TSS = 0.9982


解釈: 勉強時間で点数の変動の 99.8%を説明できる



simple-statisticsでの計算: R² = 0.9982


### 5.2.2 自由度調整済みR²

In [6]:
// 自由度調整済みR²
// Adjusted R² = 1 - (1 - R²) × (n - 1) / (n - k - 1)
// k: 説明変数の数（単回帰では1）

const k = 1;  // 説明変数の数
const adjustedRSquared = 1 - (1 - rSquared) * (n - 1) / (n - k - 1);

console.log('=== 自由度調整済みR² ===');
console.log('n =', n);
console.log('k =', k);
console.log('');
console.log('R² =', rSquared.toFixed(4));
console.log('Adjusted R² =', adjustedRSquared.toFixed(4));
console.log('');
console.log('注: 説明変数を増やすとR²は必ず増加するが、');
console.log('    Adjusted R²は不要な変数を追加すると下がる可能性がある');

=== 自由度調整済みR² ===


n = 8


k = 1


R² = 0.9982


Adjusted R² = 0.9979


注: 説明変数を増やすとR²は必ず増加するが、


    Adjusted R²は不要な変数を追加すると下がる可能性がある


### 5.2.3 標準誤差

In [7]:
// 回帰の標準誤差（残差の標準偏差）
const residualStdError = Math.sqrt(rss / (n - 2));  // 自由度: n - 2

console.log('=== 回帰の標準誤差 ===');
console.log('残差平方和 (RSS):', rss.toFixed(4));
console.log('自由度:', n - 2);
console.log('残差の標準誤差:', residualStdError.toFixed(4));
console.log('');
console.log('解釈: 予測値は平均して約', residualStdError.toFixed(2), '点ずれる可能性がある');

=== 回帰の標準誤差 ===


残差平方和 (RSS): 2.8214


自由度: 6


残差の標準誤差: 0.6857


解釈: 予測値は平均して約 0.69 点ずれる可能性がある


## 5.3 重回帰分析

複数の説明変数を使って目的変数を予測します。

$$y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_p x_p + \epsilon$$

In [8]:
// 重回帰分析の例
// 家賃を予測: 面積、築年数、駅からの距離

// データ（擬似的なサンプル）
const rentData = [
    { area: 25, age: 5, distance: 3, rent: 80 },
    { area: 30, age: 10, distance: 5, rent: 75 },
    { area: 35, age: 3, distance: 2, rent: 95 },
    { area: 40, age: 15, distance: 7, rent: 85 },
    { area: 45, age: 8, distance: 4, rent: 100 },
    { area: 50, age: 2, distance: 1, rent: 120 },
    { area: 55, age: 20, distance: 10, rent: 90 },
    { area: 60, age: 5, distance: 3, rent: 115 },
    { area: 32, age: 12, distance: 6, rent: 70 },
    { area: 48, age: 7, distance: 2, rent: 105 }
];

console.log('=== 重回帰分析: 家賃予測 ===');
console.log('説明変数: 面積(m²), 築年数(年), 駅距離(分)');
console.log('目的変数: 家賃(千円)');
console.log('');

// データを行列形式に変換
// X行列 (各行: [1, area, age, distance])、1は切片用
const X = rentData.map(d => [1, d.area, d.age, d.distance]);
const yRent = rentData.map(d => d.rent);

console.log('サンプルデータ (最初の3件):');
rentData.slice(0, 3).forEach((d, i) => {
    console.log(`  ${i+1}: 面積=${d.area}m², 築=${d.age}年, 駅=${d.distance}分 → 家賃=${d.rent}千円`);
});
console.log('  ...');

=== 重回帰分析: 家賃予測 ===


説明変数: 面積(m²), 築年数(年), 駅距離(分)


目的変数: 家賃(千円)


サンプルデータ (最初の3件):


  1: 面積=25m², 築=5年, 駅=3分 → 家賃=80千円


  2: 面積=30m², 築=10年, 駅=5分 → 家賃=75千円


  3: 面積=35m², 築=3年, 駅=2分 → 家賃=95千円


  ...


In [9]:
// 最小二乗法による重回帰係数の計算
// β = (X'X)^(-1) X'y

const XMatrix = math.matrix(X);
const yVector = math.matrix(yRent.map(y => [y]));  // 列ベクトル

// X'X (転置 × 元)
const XtX = math.multiply(math.transpose(XMatrix), XMatrix);
console.log("X'X =");
console.log(XtX.toString());

// X'y
const Xty = math.multiply(math.transpose(XMatrix), yVector);
console.log("\nX'y =");
console.log(Xty.toString());

// (X'X)^(-1)
const XtXInv = math.inv(XtX);

// β = (X'X)^(-1) X'y
const betaMatrix = math.multiply(XtXInv, Xty);
const beta = betaMatrix.toArray().map(b => b[0]);

console.log('\n=== 回帰係数 ===');
console.log('切片 (β₀):', beta[0].toFixed(4));
console.log('面積の係数 (β₁):', beta[1].toFixed(4));
console.log('築年数の係数 (β₂):', beta[2].toFixed(4));
console.log('駅距離の係数 (β₃):', beta[3].toFixed(4));

console.log('\n回帰式:');
console.log(`  家賃 = ${beta[0].toFixed(2)} + ${beta[1].toFixed(2)}×面積 + (${beta[2].toFixed(2)})×築年数 + (${beta[3].toFixed(2)})×駅距離`);

X'X =


[[10, 420, 87, 43], [420, 18828, 3710, 1823], [87, 3710, 1045, 511], [43, 1823, 511, 253]]



X'y =


[[935], [40605], [7700], [3795]]



=== 回帰係数 ===


...

面積の係数 (β₁): 1.1854


築年数の係数 (β₂): -0.5471


駅距離の係数 (β₃): -2.5074



回帰式:


  家賃 = 59.25 + 1.19×面積 + (-0.55)×築年数 + (-2.51)×駅距離


In [10]:
// 重回帰の予測と評価
function predictRent(area, age, distance) {
    return beta[0] + beta[1] * area + beta[2] * age + beta[3] * distance;
}

console.log('=== 予測と実測の比較 ===');
console.log('');

let ssRes = 0;  // 残差平方和
let ssTot = 0;  // 全変動
const yMeanRent = ss.mean(yRent);

rentData.forEach((d, i) => {
    const predicted = predictRent(d.area, d.age, d.distance);
    const residual = d.rent - predicted;
    ssRes += residual * residual;
    ssTot += Math.pow(d.rent - yMeanRent, 2);
    
    console.log(`${i+1}: 実測=${d.rent}, 予測=${predicted.toFixed(1)}, 残差=${residual.toFixed(1)}`);
});

const rSquaredMulti = 1 - ssRes / ssTot;
const adjRSquaredMulti = 1 - (1 - rSquaredMulti) * (rentData.length - 1) / (rentData.length - 3 - 1);

console.log('');
console.log('R² =', rSquaredMulti.toFixed(4));
console.log('Adjusted R² =', adjRSquaredMulti.toFixed(4));

// 新しいデータの予測
console.log('');
console.log('【新規物件の家賃予測】');
console.log('40m², 築5年, 駅から3分:', predictRent(40, 5, 3).toFixed(1), '千円');

=== 予測と実測の比較 ===


1: 実測=80, 予測=78.6, 残差=1.4


2: 実測=75, 予測=76.8, 残差=-1.8


3: 実測=95, 予測=94.1, 残差=0.9


4: 実測=85, 予測=80.9, 残差=4.1


...

10: 実測=105, 予測=107.3, 残差=-2.3


R² = 0.9533


Adjusted R² = 0.9300


【新規物件の家賃予測】


40m², 築5年, 駅から3分: 96.4 千円


## 5.4 多項式回帰

In [11]:
// 非線形な関係を持つデータ
const xPoly = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
const yPoly = [1, 4, 8, 14, 23, 35, 50, 68, 90, 115];  // おおよそ y = x² + ε

console.log('=== 多項式回帰 ===');
console.log('x:', xPoly);
console.log('y:', yPoly);
console.log('');

// 線形回帰
const linReg = ss.linearRegression(xPoly.map((x, i) => [x, yPoly[i]]));
const linPredict = ss.linearRegressionLine(linReg);
const linRSquared = ss.rSquared(xPoly.map((x, i) => [x, yPoly[i]]), linPredict);

console.log('【線形回帰】');
console.log(`y = ${linReg.b.toFixed(2)} + ${linReg.m.toFixed(2)}x`);
console.log('R² =', linRSquared.toFixed(4));
console.log('');

// 2次多項式回帰
// y = β₀ + β₁x + β₂x²
const XPoly2 = xPoly.map(x => [1, x, x * x]);
const XPoly2Matrix = math.matrix(XPoly2);
const yPolyVector = math.matrix(yPoly.map(y => [y]));

const XtXPoly2 = math.multiply(math.transpose(XPoly2Matrix), XPoly2Matrix);
const XtyPoly2 = math.multiply(math.transpose(XPoly2Matrix), yPolyVector);
const betaPoly2 = math.multiply(math.inv(XtXPoly2), XtyPoly2).toArray().map(b => b[0]);

console.log('【2次多項式回帰】');
console.log(`y = ${betaPoly2[0].toFixed(2)} + ${betaPoly2[1].toFixed(2)}x + ${betaPoly2[2].toFixed(2)}x²`);

// 2次多項式のR²
let ssResPoly2 = 0;
let ssTotPoly2 = 0;
const yMeanPoly = ss.mean(yPoly);

xPoly.forEach((x, i) => {
    const predicted = betaPoly2[0] + betaPoly2[1] * x + betaPoly2[2] * x * x;
    ssResPoly2 += Math.pow(yPoly[i] - predicted, 2);
    ssTotPoly2 += Math.pow(yPoly[i] - yMeanPoly, 2);
});
console.log('R² =', (1 - ssResPoly2 / ssTotPoly2).toFixed(4));

console.log('');
console.log('→ 2次多項式の方が当てはまりが良い');

=== 多項式回帰 ===


x: [
  1, 2, 3, 4,  5,
  6, 7, 8, 9, 10
]


y: [
   1,  4,  8, 14,  23,
  35, 50, 68, 90, 115
]


【線形回帰】


y = -27.47 + 12.41x


R² = 0.9188


【2次多項式回帰】


y = 4.53 + -3.59x + 1.45x²


R² = 0.9995


→ 2次多項式の方が当てはまりが良い


## 5.5 指数回帰と対数回帰

In [12]:
// 指数関数的な成長データ
const xExp = [1, 2, 3, 4, 5, 6, 7, 8];
const yExp = [2.5, 4.1, 6.8, 11.2, 18.5, 30.6, 50.5, 83.4];  // おおよそ y = 1.5 * 1.65^x

console.log('=== 指数回帰 ===');
console.log('データ（指数的成長）:');
xExp.forEach((x, i) => console.log(`  x=${x}: y=${yExp[i]}`));
console.log('');

// 対数変換: ln(y) = ln(a) + b*x
const lnY = yExp.map(y => Math.log(y));
console.log('対数変換後の y (ln(y)):', lnY.map(v => v.toFixed(4)));

// 線形回帰（対数変換後）
const expReg = ss.linearRegression(xExp.map((x, i) => [x, lnY[i]]));
console.log('');
console.log('対数変換後の回帰:');
console.log(`ln(y) = ${expReg.b.toFixed(4)} + ${expReg.m.toFixed(4)}x`);

// 元のスケールに戻す: y = a * e^(bx)
const a = Math.exp(expReg.b);
const b = expReg.m;
console.log('');
console.log('指数回帰式:');
console.log(`y = ${a.toFixed(4)} × e^(${b.toFixed(4)}x)`);
console.log(`y = ${a.toFixed(4)} × ${Math.exp(b).toFixed(4)}^x`);

// 予測
console.log('');
console.log('予測:');
console.log('x=10 のとき: y =', (a * Math.exp(b * 10)).toFixed(2));

=== 指数回帰 ===


データ（指数的成長）:


  x=1: y=2.5


  x=2: y=4.1


  x=3: y=6.8


...

指数回帰式:


y = 1.5092 × e^(0.5014x)


y = 1.5092 × 1.6511^x


予測:


x=10 のとき: y = 227.23


In [13]:
// べき乗回帰: y = a * x^b
const xPow = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
const yPow = [1, 2.8, 5.2, 8, 11.2, 14.7, 18.5, 22.6, 27, 31.6];  // おおよそ y = x^1.5

console.log('=== べき乗回帰 ===');
console.log('');

// 両対数変換: ln(y) = ln(a) + b*ln(x)
const lnXPow = xPow.map(x => Math.log(x));
const lnYPow = yPow.map(y => Math.log(y));

const powReg = ss.linearRegression(lnXPow.map((lnx, i) => [lnx, lnYPow[i]]));

const aPow = Math.exp(powReg.b);
const bPow = powReg.m;

console.log('べき乗回帰式:');
console.log(`y = ${aPow.toFixed(4)} × x^${bPow.toFixed(4)}`);

// 予測値との比較
console.log('');
console.log('予測値との比較:');
xPow.forEach((x, i) => {
    const predicted = aPow * Math.pow(x, bPow);
    console.log(`  x=${x}: 実測=${yPow[i]}, 予測=${predicted.toFixed(2)}`);
});

=== べき乗回帰 ===


べき乗回帰式:


y = 0.9969 × x^1.5013


予測値との比較:


  x=1: 実測=1, 予測=1.00


...

  x=6: 実測=14.7, 予測=14.69


  x=7: 実測=18.5, 予測=18.51


  x=8: 実測=22.6, 予測=22.62


  x=9: 実測=27, 予測=27.00


  x=10: 実測=31.6, 予測=31.62


## 5.6 ロバスト回帰

In [14]:
// 外れ値を含むデータ
const xRobust = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
const yRobust = [2, 4, 5, 8, 10, 12, 14, 50, 17, 20];  // 8番目のデータが外れ値

console.log('=== ロバスト回帰 ===');
console.log('外れ値を含むデータ:');
xRobust.forEach((x, i) => {
    const outlierMark = i === 7 ? ' ← 外れ値' : '';
    console.log(`  x=${x}: y=${yRobust[i]}${outlierMark}`);
});
console.log('');

// 通常の最小二乗法
const olsReg = ss.linearRegression(xRobust.map((x, i) => [x, yRobust[i]]));
console.log('【通常の最小二乗法】');
console.log(`y = ${olsReg.b.toFixed(2)} + ${olsReg.m.toFixed(2)}x`);

// Theil-Sen推定量（中央値ベースのロバスト回帰）
// すべてのペアの傾きの中央値を使用
const slopes = [];
for (let i = 0; i < xRobust.length; i++) {
    for (let j = i + 1; j < xRobust.length; j++) {
        if (xRobust[j] !== xRobust[i]) {
            slopes.push((yRobust[j] - yRobust[i]) / (xRobust[j] - xRobust[i]));
        }
    }
}

const medianSlope = ss.median(slopes);
const medianIntercept = ss.median(yRobust.map((y, i) => y - medianSlope * xRobust[i]));

console.log('');
console.log('【Theil-Sen推定（ロバスト）】');
console.log(`y = ${medianIntercept.toFixed(2)} + ${medianSlope.toFixed(2)}x`);

// 外れ値なしでの通常回帰（参考）
const xClean = xRobust.filter((_, i) => i !== 7);
const yClean = yRobust.filter((_, i) => i !== 7);
const cleanReg = ss.linearRegression(xClean.map((x, i) => [x, yClean[i]]));
console.log('');
console.log('【外れ値除去後の回帰（参考）】');
console.log(`y = ${cleanReg.b.toFixed(2)} + ${cleanReg.m.toFixed(2)}x`);

console.log('');
console.log('→ Theil-Sen推定は外れ値の影響を受けにくい');

=== ロバスト回帰 ===


外れ値を含むデータ:


  x=1: y=2


  x=2: y=4


  x=3: y=5


...

【Theil-Sen推定（ロバスト）】


y = 0.00 + 2.00x


【外れ値除去後の回帰（参考）】


y = -0.11 + 1.98x


→ Theil-Sen推定は外れ値の影響を受けにくい


## 5.7 実践例: 売上予測モデル

In [15]:
// 売上データ（広告費と売上の関係）
const salesData = [
    { tv: 230, radio: 37, newspaper: 69, sales: 22.1 },
    { tv: 44, radio: 39, newspaper: 45, sales: 10.4 },
    { tv: 17, radio: 45, newspaper: 69, sales: 9.3 },
    { tv: 151, radio: 41, newspaper: 58, sales: 18.5 },
    { tv: 180, radio: 10, newspaper: 58, sales: 12.9 },
    { tv: 8, radio: 48, newspaper: 75, sales: 7.2 },
    { tv: 57, radio: 32, newspaper: 23, sales: 11.8 },
    { tv: 120, radio: 19, newspaper: 11, sales: 13.2 },
    { tv: 199, radio: 3, newspaper: 34, sales: 10.6 },
    { tv: 66, radio: 41, newspaper: 21, sales: 13.3 },
    { tv: 214, radio: 24, newspaper: 4, sales: 17.0 },
    { tv: 284, radio: 42, newspaper: 66, sales: 24.7 },
    { tv: 50, radio: 11, newspaper: 51, sales: 8.4 },
    { tv: 164, radio: 47, newspaper: 73, sales: 19.0 },
    { tv: 67, radio: 36, newspaper: 114, sales: 12.5 }
];

console.log('=== 売上予測モデル ===');
console.log('広告費（TV, Radio, 新聞）から売上を予測');
console.log('');

// 重回帰分析
const XSales = salesData.map(d => [1, d.tv, d.radio, d.newspaper]);
const ySales = salesData.map(d => d.sales);

const XSalesMatrix = math.matrix(XSales);
const ySalesVector = math.matrix(ySales.map(y => [y]));

const XtXSales = math.multiply(math.transpose(XSalesMatrix), XSalesMatrix);
const XtySales = math.multiply(math.transpose(XSalesMatrix), ySalesVector);
const betaSales = math.multiply(math.inv(XtXSales), XtySales).toArray().map(b => b[0]);

console.log('回帰係数:');
console.log('  切片:', betaSales[0].toFixed(4));
console.log('  TV広告:', betaSales[1].toFixed(4));
console.log('  Radio広告:', betaSales[2].toFixed(4));
console.log('  新聞広告:', betaSales[3].toFixed(4));

// 予測と評価
const ySalesMean = ss.mean(ySales);
let ssResSales = 0;
let ssTotSales = 0;

salesData.forEach((d, i) => {
    const predicted = betaSales[0] + betaSales[1] * d.tv + betaSales[2] * d.radio + betaSales[3] * d.newspaper;
    ssResSales += Math.pow(d.sales - predicted, 2);
    ssTotSales += Math.pow(d.sales - ySalesMean, 2);
});

const rSquaredSales = 1 - ssResSales / ssTotSales;
const adjRSquaredSales = 1 - (1 - rSquaredSales) * (salesData.length - 1) / (salesData.length - 3 - 1);

console.log('');
console.log('モデル評価:');
console.log('  R² =', rSquaredSales.toFixed(4));
console.log('  Adjusted R² =', adjRSquaredSales.toFixed(4));

// 係数の解釈
console.log('');
console.log('【係数の解釈】');
console.log('TV広告を1単位増やすと、売上が約', betaSales[1].toFixed(3), '増加');
console.log('Radio広告を1単位増やすと、売上が約', betaSales[2].toFixed(3), '増加');
console.log('新聞広告を1単位増やすと、売上が約', betaSales[3].toFixed(3), '増加');
console.log('');
console.log('→ Radio広告が最も効果的！');

=== 売上予測モデル ===


広告費（TV, Radio, 新聞）から売上を予測


回帰係数:


  切片: 1.8798


  TV広告: 0.0560


...

【係数の解釈】


TV広告を1単位増やすと、売上が約 0.056 増加


Radio広告を1単位増やすと、売上が約 0.175 増加


新聞広告を1単位増やすと、売上が約 -0.005 増加


→ Radio広告が最も効果的！


## 5.8 練習問題

### 練習1: 単回帰分析

以下のデータで単回帰分析を行い、回帰式とR²を求めてください。

```
x = [2, 4, 6, 8, 10, 12]
y = [25, 40, 55, 70, 85, 100]
```

In [16]:
// ここに回答を書いてください



### 練習2: 多項式回帰

以下のデータに対して、線形回帰と2次多項式回帰を行い、どちらが適切か判断してください。

```
x = [1, 2, 3, 4, 5, 6, 7, 8]
y = [1, 3, 7, 13, 21, 31, 43, 57]
```

In [17]:
// ここに回答を書いてください



## まとめ

この章では以下を学びました：

1. **単回帰分析**: 最小二乗法、予測、残差
2. **モデル評価**: R²、自由度調整済みR²、標準誤差
3. **重回帰分析**: 複数の説明変数による予測
4. **多項式回帰**: 非線形関係のモデル化
5. **指数・べき乗回帰**: 対数変換による線形化
6. **ロバスト回帰**: 外れ値に強い推定

次章では、仮説検定やクラスタリングなど高度な統計解析を学びます。